In [1]:
import sys
sys.path.insert(0,'..')
sys.path.insert(0,'../../WobbleSim')
import simulacra.dataset
import astropy
import astropy.units as u 
import astropy.coordinates as coord
import astropy.time as at
import astropy.constants as const
import numpy as np

import jabble.dataset
import jabble.loss
import jabble.model

import matplotlib.pyplot as plt
from matplotlib import cm

<h1>43 - make dumb data w/ more random lines</h1>
redo the simulated data in the simplest way possible. just star no tellurics maybe gaussian noise

In [2]:
import os
import datetime
today = datetime.date.today()
out_dir = os.path.join('..','out',today.strftime("%d-%m-%y"))
os.makedirs(out_dir,exist_ok=True)

In [3]:
def gaussian(x,mu,sigma):
    return np.exp(-(x - mu)**2/(2 * sigma**2)) #/ (2 * sigma * np.sqrt(2 * np.pi))

In [4]:
resolution = 120_000
snr        = 150
epoches    = 20

dx = jabble.dataset.delta_x(resolution)
dl = 4540/resolution
print(dx,dl)
wave_grid = np.arange(4400,4440,dl) * u.Angstrom
x_grid    = np.log(wave_grid.to(u.Angstrom).value)

time = np.linspace(0,60,epoches) * u.day
period = 120 * u.day
planet_period = 2.45 * u.day

guess_vel = 60 * u.km/u.s  * np.cos(2*np.pi * time / period * u.rad)
extra_vel = 2 * u.km/u.s * np.cos(2*np.pi * time / planet_period * u.rad)
# guess_vel  = np.linspace(-60,60,epoches) * u.km/u.s
# extra_vel  = np.linspace(-10,10,epoches) * u.m/u.s
velocities = guess_vel + extra_vel
shifts     = jabble.dataset.shifts(velocities).value

# print(shifts)

mean = np.log(4550)

start_peaks = 1
end_peaks = 30
step_size = 3
peaks_vector = np.repeat(np.arange(start_peaks,end_peaks,step_size,dtype=int),repeats=5).flatten()
n_peaks = len(peaks_vector)
fluxes = np.ones((n_peaks,epoches,*wave_grid.shape))

x_padding = 3*(np.log(4551) - np.log(4550))

lines_array = np.zeros((len(peaks_vector),np.max(peaks_vector)))
for i,value in enumerate(peaks_vector):
    print(value)
    
    placements = np.random.uniform(np.min(x_grid) + np.max(shifts) + x_padding,\
                                   np.max(x_grid) - np.max(shifts) - x_padding,value)
#     placements = np.linspace(np.min(x_grid) + np.max(shifts) + x_padding,np.max(x_grid) - np.max(shifts) - x_padding,value)
    lines_array[i,:value] = placements
    for placement in placements:
        fluxes[i,:,:] -= 0.7 * gaussian(np.repeat((x_grid)[np.newaxis,:],epoches,axis=0),\
                                        np.repeat((placement + shifts)[:,np.newaxis],wave_grid.shape[0],axis=1),dx)
    
    noise = np.zeros(fluxes.shape[0:2])
    for j in range(fluxes.shape[1]):
        for k in range(fluxes.shape[2]):
            fluxes[i,j,k] -= np.random.normal(0.0,np.abs(fluxes[i,j,k]/snr))
ferr = fluxes/snr

8.333298611358605e-06 0.03783333333333333
1
1
1
1
1
4
4
4
4
4
7
7
7
7
7
10
10
10
10
10
13
13
13
13
13
16
16
16
16
16
19
19
19
19
19
22
22
22
22
22
25
25
25
25
25
28
28
28
28
28


In [16]:
import scipy.ndimage
import jax.numpy as jnp

In [17]:
datasets = []
for i in range(fluxes.shape[0]):
    datasets.append(jabble.dataset.Dataset.from_flux(np.repeat((wave_grid)[np.newaxis,:],epoches,axis=0),fluxes[i,:,:],ferr[i,:,:],mask=np.ones(fluxes[0,...].shape),normalize=scipy.ndimage.gaussian_filter,nargs=[1000]))

KeyboardInterrupt: 

In [18]:
print(len(datasets))

27


In [19]:
plt_inds = epoches
plt_ds = len(datasets)
xpmin = np.log(4550)
xpmax = np.log(4560)
fig, axes = plt.subplots(plt_ds,plt_inds,figsize=(5*plt_inds,plt_ds),sharey=True)
for ele in range(plt_inds):
#     ele = ele*3
    for ds_i in range(plt_ds):
        axes[ds_i,ele].errorbar(datasets[ds_i].xs[ele,:],datasets[ds_i].ys[ele,:],datasets[ds_i].yerr[ele,:],fmt='.k',elinewidth=2.2,zorder=1,alpha=0.5,ms=6)
#     axes[ele].set_xlim(xpmin-shifts[ele],xpmax-shifts[ele])
        axes[ds_i,ele].set_ylim(-2,0.2)
#         axes
plt.show() 

KeyboardInterrupt: 

In [20]:
loss = jabble.loss.ChiSquare()
init_vels   = guess_vel
init_shifts = jabble.dataset.shifts(init_vels)

In [21]:
class MyModel(jabble.model.JaxLinear):
    def __init__(self,*args):
        super(MyModel,self).__init__(*args)
        self.p_i = None
    
    
    def split_p(self,p,*args):
        if len(p) !=  0:
            self.p = p
    
    def fit(self):
        self.p_i  = None
        self._fit = True
    
    def fix(self):
        self.p_i  = None
        self._fit = False
           
    def fit_p(self,p_i):
        self.p_i  = p_i
        self._fit = True
        
    def get_parameters(self):
        if self._fit:
            if self.p_i is not None:
                return jnp.array(self.p[self.p_i])
            return jnp.array(self.p)
        else:
            return jnp.array([])
        
    def unpack(self,p):
        if len(p) != 0:
            if self.p_i is not None:
                self.p[self.p_i] = p
            else:
                self.p = p
          
    def __call__(self,p,*args):
#         print('on call')
        # if there are no parameters coming in, then use the stored parameters
        if len(p) == 0:
            assert self._fit == False
            return self.call(self.p,*args)
        else:
            assert self._fit == True
            if self.p_i is not None:
                return self.call(jnp.array(self.p).at[self.p_i].set(p),*args)
            return self.call(p,*args)

In [22]:
vel_padding = 100 * u.km/u.s
model_grid = jabble.model.create_x_grid(x_grid,vel_padding,resolution)#step=jabble.dataset.shifts(const.c.to(u.km/u.s)/resolution))
model_list = []
for i in range(n_peaks):
    model_list.append(jabble.model.CompositeModel([jabble.model.ShiftingModel(init_shifts),MyModel(model_grid)]))

In [23]:
model_grid.shape

(1166,)

In [24]:
def train_cycle(model,dataset,loss,fit1_inds,fit2_inds):
    options = {"maxiter":15000,'maxls':256,'ftol':1e-40,'gtol':1e-35,'tol':1e-30,'eps':1e-22}
    
    # Fitting Round 1
    print('fitting round 1')
    model.fix()
    model.fit(1)
    res1 = model.optimize(loss,dataset,options=options,verbose=False)
    print(res1['message'],res1['nit'])
    
    # Fitting Round 2
    print('fitting round 2')
    model.fix()
    model.fit(0)
    res2 = model.optimize(loss,dataset,options=options,verbose=False)
    print(res2['message'],res2['nit'])
    
    # Fitting Round 3
    print('fitting round 3')
    model.fix()
    model.fit(1)
    res3 = model.optimize(loss,dataset,options=options,verbose=False)
    print(res3['message'],res3['nit'])
    
    # Fitting Round 4
    print('fitting round 4')
    model.fix()
    model.fit(0)
    model.fit(1)
    res4 = model.optimize(loss,dataset,options=options,verbose=False)
    print(res4['message'],res4['nit'])

#     # Fitting Round 5
#     print('fitting round 5')
#     model.fix()
#     model.fit(0)
#     model.fit(1)
#     res5 = model.optimize(loss,dataset,options=options,verbose=True)
#     print(res5['message'],res5['nit'])
    return model

In [25]:
ind1 = [[1]]
ind2 = [[0],[1]]

for i,model in enumerate(model_list):
    print('model #{}'.format(i))
    model_list[i] = train_cycle(model,datasets[i],loss,ind1,ind2)

model #0
fitting round 1
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH 17
fitting round 2
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH 7
fitting round 3
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH 30
fitting round 4
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH 3
model #1
fitting round 1
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH 17
fitting round 2
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH 4
fitting round 3
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH 21
fitting round 4
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH 5
model #2
fitting round 1
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH 14
fitting round 2
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH 8
fitting round 3
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH 23
fitting round 4
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH 4
model #3
fitting round 1
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH 15
fitting round 2
CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH 8
fitting round 3
CONVERGENCE: REL_

IndexError: list index out of range

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
velocities.to(u.m/u.s)

In [ ]:
# plt_ds = n_peaks
# fig,axes = plt.subplots(plt_ds,figsize=(5,10),sharex=True,sharey=True)

# for i in range(plt_ds):
#     axes[i].plot(extra_vel.to(u.m/u.s).value,'.g')
#     axes[i].plot(jabble.dataset.velocities(model_list[i][0].p).to(u.m/u.s).value + init_vels.to(u.m/u.s).value,'.r')
#     axes[i].set_ylabel('# {}'.format(i))
#     # axes.supylabel('dv [m/s]')
# plt.xlabel('epoches')
# plt.show()

In [ ]:
def norm_func(y,sigma):
    y_low = scipy.ndimage.gaussian_filter(y,sigma)
    return y_low/y

In [ ]:
n_peaks

In [ ]:
# for i,model in enumerate(model_list):
#     print('model #{}'.format(i))
# #     print(model.results)
#     for result in model.results:
#         print(result['message'])
#         print(result['nit'])

In [ ]:
lmin = 4397
lmax = 4415
x_window = np.log(4551) - np.log(4550.5)

lrange = np.arange(lmin,lmax,1)
plt_unit = u.Angstrom
plt_epoch = epoches
plt_ds = len(datasets)
fig, axes = plt.subplots(plt_ds,plt_epoch,figsize=(4,8),sharex=False,sharey=True,facecolor=(1, 1, 1),dpi=100)
for i in range(plt_ds):
    for j in range(plt_epoch):
        axes[i,j].set_xlim(np.log(lmin),np.log(lmax))
    #         datasets[i].plot_star(axes[i,j],j,xy='xy',units=plt_unit,normalize=normalize_2,nargs=[sigma])
    #     data.plot_tellurics(axes[i],i,xy='xy',units=plt_unit)
        axes[i,j].errorbar(datasets[i].xs[j,:],datasets[i].ys[j,:],datasets[i].yerr[j,:],fmt='.k',elinewidth=2.2,zorder=3,alpha=0.8,ms=6)
        axes[i,j].plot(model_list[i][1].xs+model_list[i][0].p[j],model_list[i][1].p,'.r',linestyle='solid',linewidth=2.2,zorder=1,alpha=0.5,ms=6)
        
        
        
        axes[i,j].set_ylim(-1.4,0.2)
        
        axes[i,j].set_yticks([])
        axes[i,j].set_xticks(np.log(lrange))
        axes[i,j].set_xticklabels(['{:2.0f}'.format(x) for x in lrange])
        axes[i,j].set_xlim(lines_array[i,0]-x_window+model_list[i][0].p[j],\
                           lines_array[i,0]+x_window+model_list[i][0].p[j])
        res_ax = axes[i,j].twinx()
        res_ax.set_yticks([])
        residual = loss(model_list[i].get_parameters(),datasets[i],j,model_list[i])
#         res_ax.step(datasets[i].xs[j,:],residual,where='mid',alpha=0.3,label='residual')
        res_ax.step(model_list[i][1].xs+model_list[i][0].p[j],\
                model_list[i].results[-1]['jac'][model_list[i][0].p.shape[0]:],\
                where='mid',alpha=0.4,label='gradient',zorder=-1)
#         res_ax.set_xlim(xpmin,xpmax)
#         res_ax.set_ylim(-1e2,1e2)
    #     axes[j].plot(np.log(data['theory']['interpolated']['total']['wave'].to(u.Angstrom).value),np.log(norm_func(data['theory']['interpolated']['star']['flux'][j,:],sigma)))
    
plt.savefig(os.path.join(out_dir,'43-allplots.png'),dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
plt_ds = n_peaks
fig,ax = plt.subplots(1,figsize=(8,5),sharex=True,sharey=True,facecolor=(1,1,1),dpi=300)
estimate_vel = np.zeros((plt_ds,epoches))
for i in range(plt_ds):
    estimate_vel[i,:] = jabble.dataset.velocities(model_list[i][0].p).to(u.m/u.s).value

for i in range(plt_ds):
    
    ax.plot(estimate_vel[i,:],'.',color=cm.plasma(i/n_peaks))
#     ax.set_ylabel('# {}'.format(i))
ax.plot((velocities.to(u.m/u.s).value),'.g',alpha=0.2)
plt.ylabel('vel [m/s]')
plt.xlabel('time')
plt.savefig(os.path.join(out_dir,'43-vel.png'))
plt.show()

In [ ]:
# plt_ds = n_peaks
fig,ax = plt.subplots(1,figsize=(8,5),sharex=True,sharey=True,facecolor=(1,1,1),dpi=200)

for i,num_peaks in enumerate(peaks_vector):
#     ax.plot(,'.g')
    symb = '.'
    if model_list[i].results[1]['message'] == 'ABNORMAL_TERMINATION_IN_LNSRCH':
        symb = '*'
    ax.plot(time%planet_period, velocities.to(u.m/u.s).value - \
            estimate_vel[i,:],\
            symb,color=cm.plasma(num_peaks/np.max(peaks_vector)),\
            label='{} lines'.format(num_peaks))
#     ax.set_ylabel('# {}'.format(i))
ax.set_ylim(-100,100)
ax.plot(time%planet_period,(extra_vel.to(u.m/u.s).value),'og',label='initialization',alpha=0.2)
# plt.legend()
plt.ylabel('dv [m/s]')
plt.xlabel('time mod planet period')
plt.savefig(os.path.join(out_dir,'43-dv.png'))
plt.show()

$$RMS(dv) = \sqrt{\frac{1}{N} \sum_{i=0}^N (\frac{(\hat{v}^i - v^i_{true})}{v^i_{true}})^2}$$

In [ ]:
def RMS_DV(est_vel,true_vel):
    return np.sqrt((((est_vel - true_vel))**2).mean())

In [ ]:
plt_ds = n_peaks
fig,ax = plt.subplots(1,figsize=(8,5),sharex=True,sharey=True,dpi=200,facecolor=(1,1,1))
# peak_vector = np.arange(2,n_peaks+2)
dv_vector = np.zeros(peaks_vector.shape)

worst_dv = RMS_DV(guess_vel.to(u.m/u.s).value,velocities.to(u.m/u.s).value)
# plt.hlines([worst_dv],np.min(peaks_vector),np.max(peaks_vector),'m',alpha=0.4)

for i in range(plt_ds):
    dv_vector[i] = RMS_DV(estimate_vel[i,:] - estimate_vel[i,:].mean(), velocities.to(u.m/u.s).value - velocities.to(u.m/u.s).value.mean())
ax.plot(peaks_vector,dv_vector,'.g')
plt.ylabel('RMS(dv)')
plt.xlabel('# of lines')
plt.ylim(0,200)
plt.savefig(os.path.join(out_dir,'43-rmsdv.png'))
plt.show()

In [ ]:
for i,model in enumerate(model_list):
#     print('model #{}'.format(i))
#     print(model.results)
#     for result in model.results:
#         print(result['message'])
#         print(result['nit'])
    if model.results[1]['message'] == 'ABNORMAL_TERMINATION_IN_LNSRCH':
        print(i)

In [ ]:
extra_vel.to(u.m/u.s).value.mean()

In [ ]:
velocities.to(u.m/u.s).value.mean()

In [ ]:
guess_vel.to(u.m/u.s).value.mean()

In [ ]:
for i in range(len(datasets)):
    print(estimate_vel[i,:].mean())